# Statistiques

Aux chapitres précédents, nous avons vu comment organiser et manipuler des tableaux de données, puis les visualiser. La statistique est une collection de disciplines liées à la collecte, l'oragnisation, l'analyse, l'interprétation et la présentation de données. Les biostatistiques est l'application de ces disciplines à la biosphère.

Les statistiques consistent à "faire du sens" (anglicisme assumé) avec des observations: collecter des échantillons, transformer les données, effectuer des tests,  analyser les résultats, les interpréter et les visualiser. Bien que ces tâches soient complexes, en particulier en ce qui a trait aux tests statistiques, la plupart des opérations statistiques peuvent être effectuées sans l'assistance de statisticien.ne.s. Les erreurs d'interprétation statistiques sont néanmoins courantes et la consultation de spécialistes n'est souvent pas un luxe.

Dans ce chapitre, nous verrons comment répondre correctement à une question valide et adéquate avec l'aide d'outils de calcul scientifique. Nous couvrirons les notions de bases des distributions et des variables aléatoires qui nous permettront d'effectuer des tests statistiques commun avec Python. Nous couvrirons aussi les erreurs communément commises en recherche académique et les moyens simples de les éviter.

Ce chapitre est une introduction aux statistiques avec Python, et ne rempalcera pas un bon cours de stats.

Nous aurons besoin des modules suivants.

- Modules scientifiques de base: `numpy`, `scipy` et `pandas`
- Modules graphiques: `matplotlib` et `seaborn`
- Modules de calcul statistique: [`statsmodels`](http://www.statsmodels.org).

## Poser la bonne question

La conception d'une expérience commence par une question. Ce chapitre est destinées à la statistique appliquée: il ne comprend pas la phase de conception expérimentale, une discipline a part entière. Les principes d'expérimentation sont plutôt présentés dans le cours [*Dispositifs expérimentaux (BVG-7002)*](https://www.ulaval.ca/les-etudes/cours/repertoire/detailsCours/bvg-7002-dispositifs-experimentaux.html). Également, je recommande le livre *Princpes d'expérimentation: planification des expériences et analyse de leurs résultats* de Pierre Dagnelie (2012), [disponible en ligne en format PDF](http://www.dagnelie.be/docpdf/ex2012.pdf).

- Quels sont les variables et leurs types
- Quels effets désire-t-on évaluer, de quel type de variable s'agit-il


...

## Pile ou face?

Débuttons par un exemple classique tiré d'une [conférence de Jake Vanderplas](https://www.youtube.com/watch?v=Iq9DzN6mvYA). Lorsque l'on lance une pièce de monnaie, le résultat est soit pile, soit face. Si la pièce n'est pas biaisée, la probabilité d'obtenir l'un ou l'autre de ces résultats est égale. `numpy` comprend des algorithmes pour générer des nombres aléatoires selon des distributions. Dans le cas d'un jeu de hasard à deux possibilités, on parle d'une distribution binomiale: on utilise la fonction `np.random.binomial` pour générer des tirages aléatoires. On peut lancer une pièce virtuelle avec Python. Mais avant tout, appelons `numpy`, `pandas` et `matplotlib.pyplot`.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

*Mise en situation*: Vous jouez à pile ou face avec Schtroumpfette. Ce sera elle ou vous qui devrez payer un sac de chip au gagnant au jeu de pile ou face où la pèce est lancée 25 fois. Vous choisissez pile, elle choisi face. La fonction `np.random.binomial` retournera le nombre de 1 (pile). Quant à la fonction `np.random.seed`, elle assume que la sortie de la fonction aléatoire sera toujours la même à chaque fois que la cellule sera exécutée (pour les fins de l'exemple).

In [8]:
np.random.seed(seed=184008)
pf_result = np.random.binomial(n=25, p=0.5)
pf_result

19

Fantastique, vous avez 19 lancés à votre faveur: vous vous pressez de commissionner Schtroumpfette au dépanneur pour un sac de chips aux baies schtroumpfantes. Mais Schtroumpfette n'est pas d'accord: pour elle, un tel score ne peut pas avoir été le résultat d'une pièce non biaisée. Vous pourriez alors demander au Schtroumpf ingénieur d'examiner la distribution de la masse dans la pièce: mais ce serait une approche mécanistique laborieuse, et il s'avère que Schtroumpfette, qui est ferrée en stats, y va de manière analytique.

La probabilité d'obtenir l'un ou l'autre des résultats (disons face, ou $F$) est de 50%, donc $P\left( F \right) = \frac{1}{2}$. La probabilité d'obtenir deux $F$ consécutifs est de $P\left( FF \right) = \left( \frac{1}{2} \right)^2$.

La probabilité d'obtenir face, face, pile est de $P\left( FF \right) = \left( \frac{1}{2} \right)^3$. Mais l'ordre dans lequel les résultats sont obtenu n'est pas important dans la question. Alors, la probabilité d'obtenir 2 face et un pile est de $P\left( 2F, 1P \right) = P\left( FFP \right) + P\left( FPF \right) + P\left( PFF \right) = \frac{3}{8}$.

Schtroumpfette connait la fonction d'une distribution binômiale, qui permet de généraliser le calcul de probabilité lorsque l'ordre n'est pas important, avec $k$ succès d'une probabilité $p$ avec $n$ tirages.

$$P\left(n, k\right) = \binom{n}{k} p^k \left( 1-p \right)^{n-k}$$

où 

$$\binom{n}{k} = \frac{n!}{k!\left( n-k\right)!}$$

où $n! = n \times \left(n-1 \right) \times ... \times 2 \times 1$.

Schtroumpfette recherche la probabilité d'obtenir un tel résultat en supposant que la pièce n'est pas biaisée (i.e. que la probabilités d'obtenir face ou pile est de 0.5). Vous commencez à être agile en Python et vous encodez la formule que Schtroumpfette vous transmet.

In [72]:
def my_binomial(n, k, p):
    nk = np.math.factorial(n) / (np.math.factorial(k) * np.math.factorial(n-k))
    prob = nk * p**k * (1-p)**(n-k)
    return prob

Vous calculez les probabilités d'obtenir des résultats de 19 à 25 inclusivement, puis vous en faites la somme.

In [73]:
probs = []
for i in range(19, 26): # avec range, la limite supérieure est exclue
    probs.append(my_binomial(25, i, 0.5))
    
print("La probabilité d'obtenir 19 piles ou plus est de",
       np.sum(probs) * 100, "%") 

La probabilité d'obtenir 19 piles ou plus est de 0.731664896011 %


Vous venez de calculer une **p-value**.

Une **p-value** est une probabilité que le résultat soit obtenu de manière aléatoire si l'hypothèse nulle est vraie.

Dans le cas qui nous concerne, il s'agit de la probabilité d'obtenir 19 succès ou plus dans un jeu non biaisé. La convention veut que le seuil de significativité d'un test en biostatistique soit de 5%. Dans ce cas particulier, la probabilité de 0.7% est plus faible que 5% et nous pouvons conclure à ce seuil que la pièce est biaisée. 

Nous y reviendrons, mais retenez pour l'instant que la **p-value** est dans la littérature scientifique une source importante de mauvaise interprétation et que le seuil purement arbitraire de 5% n'est pas un seuil particulièrement prudent.

Puisque vous maîtrisez bien Python, vous désirez éviter les formulations statistiques. Vous modélisez alors un grand nombre de jeux de 25 lancés que vous savez non-biaisés (avec `numpy`), puis vous calculez la proportion de ces lancés qui donnent 19 piles ou plus. Comme ceci.

In [74]:
more_than_19 = 0 # initialisation du décompte des jeux où vous gagnez avec 19 et plus
n_sim = 100000 # nombre de simulations
for i in range(n_sim):
    pf_result = np.random.binomial(n=25, p=0.5) # tirage
    if pf_result >= 19: # si le tirage donne 19 ou plus, ajouter 1 au décompte
        more_than_19 = more_than_19 + 1

print("La proportion des", n_sim, 
      "simulations ayant obtenu 19 piles ou plus est de",
      more_than_19/n_sim * 100, "%")

La proportion des 100000 simulations ayant obtenu 19 piles ou plus est de 0.7020000000000001 %


Le résultat est semblable. Pour une personne habituée à réfléchir en terme de code, les tenants et aboutissants du processus par modélisation sont bien plus intuitif qu'un développement analytique. La formulation analytique reste néanmoins la référence en science.

## Les probabilités
> « Nous sommes si éloignés de connaître tous les agens de la nature, et leurs divers modes d'action ; qu'il ne serait pas philosophique de nier les phénomènes, uniquement parce qu'ils sont inexplicables dans l'état actuel de nos connaissances. Seulement, nous devons les examiner avec une attention d'autant plus scrupuleuse, qu'il paraît plus difficile de les admettre ; et c'est ici que le calcul des probabilités devient indispensable, pour déterminer jusqu'à quel point il faut multiplier les observations ou les expériences, afin d'obtenir en faveur des agens qu'elles indiquent, une probabilité supérieure aux raisons que l'on peut avoir d'ailleurs, de ne pas les admettre. » — Pierre-Simon de Laplace

Une probabilité est la vraissemblance qu'un événements se réalise chez un échantillon. Les probabilités forment le cadre des systèmes stochastiques, c'est-à-dire des systèmes trop complexes pour en connaître exactement les aboutissants, auxquels ont attribue une part de hasard. Ces systèmes sont prédominants dans les processus vivants.

On peut dégager deux perspectives sur les probabilités: l'une passe par une interprétations fréquentistes, l'autre bayésiennes. L'interprétation **fréquentiste** représente la fréquence des occurences après un nombre infini d'événements. Par exemple, si vous jouez à pile ou face un grand nombre de fois, le nombre de pile sera égal à la moitié du nombre de lancés. Il s'agit de l'interprétation communément utilisée.

L'interprétation **bayésienne** vise à quantifier l'incertitude des phénomènes. Dans cette perspective, plus l'information s'accumule, plus l'incertitude diminue. Cette approche gagne en notoriété notamment parce qu'elle permet de décrire des phénomènes qui, intrinsèquement, ne peuvent être répétés infiniments (absence d'asymptote), comme celles qui sont bien définis dans le temps ou sur des populations limités.

Bien que les statistiques bayésiennes soient de plus en plus utilisées, nous ne couvrirons dans ce chapitre que l'approche fréquentielle. L'approche baésienne est néamoins traitée dans le document compplémentaire 5.1_bayes.ipynb.

## Les distributions
Une distribution est une fonction mathématique décrivant la probabilité d'observer une série d'évennements. Ces événemements peuvent être des valeurs continues, des nombres entiers, des catégories, des valeurs booléennes (Vrai/Faux), etc. Dépendamment du type de valeur et des observations obtenues, on peut associer des variables à différentes lois de probabilité.

### Normale / Student
La plus répendue de ces lois est probablement la loi normale, parfois nommée loi gaussienne. Il s'agit de la distribution classique en forme de cloche. Lorsque l'on a peut d'observations, en général moins de 30, la loi normale sera avantageusement substituée par la loi de Student. Au delà de 30, la loi de Student et la loi normale sont équivalentes.

La loi normale est décrite par une moyenne, qui désigne la tendance centrale, et une variance, qui désigne l'étalement des probabilités autours de la moyenne. La racine carrée de la variance est l'écart-type.

En analyse multivariée, des données distribuées de manière multinormale ont des distributions normales sous toutes les perspectives. Elles sont désignées par un vecteur de moyennes et une matrice de covariance.

Pour les distributions mesures exclusivement positives (comme le poids ou la taille) sont parfois avantageusement approximées par une loi log-normale, ce qui est une loi normale sur le logarithme des valeurs: la moyenne d'une loi log-normale est la moyenne géométrique.

### Uniforme
### Binomiale
### Poisson
### Exponentielle
### Beta

## Tests statistiques

### Variance, covariance et corrélation

### Test de t

### ANOVA
- Anova sens unique
- Anova double sens

### MANOVA et MANCOVA
Tout comme les ANOVA et les ANCOVA supposent que les les données sont distribuées normalement, les MANOVA et MANCOVA supposent des données multinormales. La multinornalité des données peut être testée avec un test de M (Box's M test).

### Modélisation linéaire
- GLM
- dummification des facteurs et matrices de concept
- Régression linéaire (lmplot, residplot)



### Analyse de covariance


### Modélisation mixte

L'outil que je privilégie en biostatistique est le modèle linéaire mixte, qui consiste à intégrer dans un modèle linéaire autant des effets fixes que des effets aléatoires.

## Erreurs communes

### Un résultat montrant une p-value plus élevée que 0.05 n'est pas pertinent

Lors d'une conférence, Dr Evil ne présentent que les résultats significatifs de ses essais au seuil de 0.05. Certains essais ne sont pas significatifs, mais bon, ceux-ci ne sont pas importants... En écartant ces résultats, Dr Evil commet 3 erreurs:

1. La p-value n'est pas un bon indicateur de l'importance d'un test statistique. L'importance d'une variable dans un modèle linéaire devrait être évaluée par la valeur de son coefficient. Son incertitude devrait être évaluée par sa variance. Une manière d'évaluer plus intuitive la variance est l'écart-type ou l'intervalle de confiance. À un certain seuil d'intervalle de confiance, la p-value traduira la probabilité qu'un coefficient soit réellement nul ait pu générer des données démontrant un coefficient égal ou supérieur.
2. Il est tout aussi important de savoir que le traitement fonctionne que de savoir qu'il ne fonctionne pas. Les résultats démontrant des effets sont malheureusement davantage soumis aux journaux et davantage publiés que ceux ne démontrant pas d'effets ([Decullier et al., 2005]( https://doi.org/10.1136/bmj.38488.385995.8F )).
3. Le seuil de 0.05 est arbitraire.

